In [13]:
# Base de Conocimiento
KB = {
    "Hechos": [
        
    ],
    "Reglas": [
        {
            "condicion": ["riesgo=bajo", "eficiencia_energetica=True"], 
            "decision": "Aprobado"
        },
        {
            "condicion": ["riesgo=alto", "impacto_ambiental=negativo"],
            "decision": "Rechazado"
        },
        {
            "condicion": ["endeudamiento=medio", "impacto_ambiental=moderado"],
            "decision": "AprobarCondicionadoGarantia"
        }
    ],
    "ReglasCombinadas": [
        {
            "condicion": ["riesgo=medio", "impacto_ambiental=positivo", "cofinanciamiento=True"], 
            "decision": "AprobadoCondicionadoMitigaciónSeguimiento",
            "FactoresAdicionales": ["ApoyoComunitario"]
        },
        {
            "condicion":["riesgo=alto", "eficiencia_energetica=False", "impacto_ambiental=negativo"],
            "decision": "RechazadoDefinitivo",
            "FactoresAdicionales": ["RechazoComunitario"]
        },
    ]
}

In [14]:
# Recopilación de Datos
def recopilar_datos(info_cliente):
    hechos = {info: True for info in info_cliente}
    print("Datos recopilados:", info_cliente)
    return hechos

In [15]:
# Aplicación de Reglas Específicas
def aplicar_reglas_especificas(KB, hechos):
    resultado = []
    for i, regla in enumerate(KB["Reglas"], 1):
        if verificar_condicion(hechos, regla["condicion"]):
            resultado.append({
                "decision": regla["decision"],
                "condicion": regla["condicion"],
                "regla_num": i
            })
            print(f"Regla específica #{i} activada: condición = {regla['condicion']} => decisión = {regla['decision']}")
    return resultado

In [16]:
# Aplicación de Reglas Combinadas
def aplicar_reglas_combinadas(KB, hechos, factores_adicionales):
    resultados_combinados = []
    for regla in KB["ReglasCombinadas"]:
        if (verificar_condicion(hechos, regla["condicion"]) and
                verificar_factores(factores_adicionales, regla["FactoresAdicionales"])):
            resultados_combinados.append(regla["decision"])
            print(f"Regla combinada activada: {regla['decision']}")
    return resultados_combinados

In [17]:
# Funciones Auxiliares
def verificar_condicion(hechos, condicion):
    for condic in condicion:
        if condic not in hechos:
            return False
    return True

def verificar_factores(factores_adicionales, factores_requeridos):
    # Solo verifica si el nombre del factor está presente en algún hecho
    for factor in factores_requeridos:
        if not any(factor.lower() in h.lower() for h in factores_adicionales):
            return False
    return True

In [18]:
def priorizar_decisiones(decisiones, solicitante, proyecto):
    """
    Prioriza las decisiones de crédito basándose en un sistema de puntajes
    """
    return sorted(decisiones, key=lambda x: calcular_puntaje_decision(x, solicitante, proyecto), reverse=True)

def calcular_puntaje_decision(decision, solicitante, proyecto):
    """
    Calcula un puntaje para priorizar decisiones de crédito
    """
    puntaje = 0
    
    # Bonificaciones adicionales basadas en el historial del solicitante
    if tiene_historial_positivo(solicitante):
        puntaje += 15
    
    # Bonificación por tipo de proyecto
    if es_proyecto_prioritario(proyecto):
        puntaje += 10
    
    # Bonificación por factores ambientales
    if tiene_impacto_ambiental_positivo(solicitante, proyecto):
        puntaje += 8
    
    return puntaje

def tiene_historial_positivo(solicitante):
    """
    Verifica si el solicitante tiene un historial crediticio positivo
    """
    # Buscar en los hechos del solicitante por indicadores positivos
    hechos_solicitante = [h for h in KB["Hechos"] if h["solicitante"] == solicitante]
    
    for hecho in hechos_solicitante:
        if (hecho.get("condicion") == "riesgo" and hecho.get("valor") in ["bajo", "medio"]) or \
           (hecho.get("condicion") == "historial_crediticio" and hecho.get("valor") == "bueno"):
            return True
    return False

def es_proyecto_prioritario(proyecto):
    """
    Determina si el proyecto es de alta prioridad ambiental
    """
    proyectos_prioritarios = ["Planta Solar", "Edificio Verde", "Parque Eólico", "Sistema Geotérmico"]
    return any(prioritario.lower() in proyecto.lower() for prioritario in proyectos_prioritarios)

def tiene_impacto_ambiental_positivo(solicitante, proyecto):
    """
    Verifica si el proyecto tiene impacto ambiental positivo
    """
    hechos_proyecto = hechos_de(solicitante, proyecto)
    
    for hecho in hechos_proyecto:
        if (hecho.get("condicion") == "impacto_ambiental" and hecho.get("valor") == "positivo") or \
           (hecho.get("condicion") == "eficiencia_energetica" and hecho.get("valor") is True):
            return True
    return False

In [19]:

def agregar_hecho(solicitante, proyecto, condicion, valor):
    KB["Hechos"].append({
        "solicitante": solicitante,
        "proyecto": proyecto,
        "condicion": condicion,
        "valor": valor
    })

In [20]:
def hechos_de(solicitante, proyecto):
    return [h for h in KB["Hechos"] if h["solicitante"] == solicitante and h["proyecto"] == proyecto]

In [21]:
def evaluar_credito(info_cliente):
    # Recopilación de datos
    hechos = recopilar_datos(info_cliente)

In [22]:
print(KB)

{'Hechos': [], 'Reglas': [{'condicion': ['riesgo=bajo', 'eficiencia_energetica=True'], 'decision': 'Aprobado'}, {'condicion': ['riesgo=alto', 'impacto_ambiental=negativo'], 'decision': 'Rechazado'}, {'condicion': ['endeudamiento=medio', 'impacto_ambiental=moderado'], 'decision': 'AprobarCondicionadoGarantia'}], 'ReglasCombinadas': [{'condicion': ['riesgo=medio', 'impacto_ambiental=positivo', 'cofinanciamiento=True'], 'decision': 'AprobadoCondicionadoMitigaciónSeguimiento', 'FactoresAdicionales': ['ApoyoComunitario']}, {'condicion': ['riesgo=alto', 'eficiencia_energetica=False', 'impacto_ambiental=negativo'], 'decision': 'RechazadoDefinitivo', 'FactoresAdicionales': ['RechazoComunitario']}]}


In [ ]:

def main():
    KB["Hechos"] = []
    # Datos de prueba
    datos = [
        {"solicitante": "Juan", "proyecto": "Edificio Verde", "hechos": [
            ("riesgo", "bajo"),
            ("eficiencia_energetica", True)
        ]},
        {"solicitante": "Ana", "proyecto": "Fabrica", "hechos": [
            ("riesgo", "alto"),
            ("impacto_ambiental", "negativo")
        ]},
        {"solicitante": "Luis", "proyecto": "Planta Solar", "hechos": [
            ("riesgo", "medio"),
            ("impacto_ambiental", "positivo"),
            ("cofinanciamiento", True),
            ("ApoyoComunitario", None)
        ]}
    ]

    for d in datos:
        for cond, val in d["hechos"]:
            agregar_hecho(d["solicitante"], d["proyecto"], cond, val)

    for d in datos:
        hechos = hechos_de(d["solicitante"], d["proyecto"])
        hechos_str = [f"{h['condicion']}={h['valor']}" for h in hechos]
        print(f"\n--- {d['solicitante']} / {d['proyecto']} ---")
        res_especificas = aplicar_reglas_especificas(KB, hechos_str)
        res_combinadas = aplicar_reglas_combinadas(KB, hechos_str, hechos_str)
        print("Reglas específicas:")
        if res_especificas:
            for r in res_especificas:
                print(f"  Regla #{r['regla_num']}: condición = {r['condicion']} => decisión = {r['decision']}")
        else:
            print("  Ninguna regla específica aplicada.")
        print("Reglas combinadas:")
        if res_combinadas:
            for rc in res_combinadas:
                print(f"  Decisión combinada: {rc}")
        else:
            print("  Ninguna regla combinada aplicada.")


definir = main()


--- Juan / Edificio Verde ---
Regla específica #1 activada: condición = ['riesgo=bajo', 'eficiencia_energetica=True'] => decisión = Aprobado
Reglas específicas:
  Regla #1: condición = ['riesgo=bajo', 'eficiencia_energetica=True'] => decisión = Aprobado
Reglas combinadas:
  Ninguna regla combinada aplicada.

--- Ana / Fabrica ---
Regla específica #2 activada: condición = ['riesgo=alto', 'impacto_ambiental=negativo'] => decisión = Rechazado
Reglas específicas:
  Regla #2: condición = ['riesgo=alto', 'impacto_ambiental=negativo'] => decisión = Rechazado
Reglas combinadas:
  Ninguna regla combinada aplicada.

--- Luis / Planta Solar ---
Regla combinada activada: AprobadoCondicionadoMitigaciónSeguimiento
Reglas específicas:
  Ninguna regla específica aplicada.
Reglas combinadas:
  Decisión combinada: AprobadoCondicionadoMitigaciónSeguimiento
